In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### Reading Train and Test data sets from CSV file

In [33]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
gs = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

### Printing Values of Train and Test data

In [34]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Checking Empty and Null cells in Columns

In [36]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [37]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

### Dropping some unused columns from dataframes

In [38]:
train_data = train_data.drop(columns=['Name','Ticket','Fare','Cabin','PassengerId','Age'])
test_data = test_data.drop(columns=['Name','Ticket','Fare','Cabin','PassengerId','Age'])

### Replacing Missing values in for Embarked columns

In [39]:
train_data.Embarked = train_data.Embarked.fillna(train_data['Embarked'].mode()[0])

### Converting dataframes to numpy arrays

In [40]:
X_train_data = train_data[['Pclass','Sex','SibSp','Parch','Embarked']].to_numpy()
y_train_Data = train_data['Survived'].to_numpy()
X_test_data = test_data[['Pclass','Sex','SibSp','Parch','Embarked']].to_numpy()
y_test_data = gs['Survived'].to_numpy()

### Feature Encoding using OneHotEncoder - Train and Test data sets

In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X_train_data = np.array(ct.fit_transform(X_train_data))

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[-1])],remainder='passthrough')
X_train_data = np.array(ct.fit_transform(X_train_data))

In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X_test_data = np.array(ct.fit_transform(X_test_data))

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[-1])],remainder='passthrough')
X_test_data = np.array(ct.fit_transform(X_test_data))

### Applying Standard scaling

In [43]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_data[1:2,5:7] = sc.fit_transform(X_train_data[1:2,5:7])
X_test_data[1:2,5:7] = sc.transform(X_test_data[1:2,5:7])

### Logistic Regression - Confusion Matrix and Accuracy score

In [44]:
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression(max_iter=300)
regressor.fit(X_train_data,y_train_Data)

LogisticRegression(max_iter=300)

In [45]:
y_pred = regressor.predict(X_test_data)
from sklearn.metrics import confusion_matrix,accuracy_score
logistic_confusionMatrix = confusion_matrix(y_pred,y_test_data)
logistic_Accuracy = accuracy_score(y_pred,y_test_data)

### KNN Classifier - Confusion Matrix and Accuracy score

In [46]:
from sklearn.neighbors import KNeighborsClassifier
knnclassifier = KNeighborsClassifier(n_neighbors=5)
knnclassifier.fit(X_train_data,y_train_Data)

KNeighborsClassifier()

In [47]:
y_pred_knn = knnclassifier.predict(X_test_data)
knn_confusionMatrix = confusion_matrix(y_pred_knn,y_test_data)
knn_Accuracy = accuracy_score(y_pred_knn,y_test_data)

### Support Vector Classifier - Confusion Matrix and Accuracy score

In [48]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf')
svr.fit(X_train_data,y_train_Data)

SVR()

In [49]:
y_pred_svr = knnclassifier.predict(X_test_data)
svr_confusionMatrix = confusion_matrix(y_pred_svr,y_test_data)
svr_Accuracy = accuracy_score(y_pred_svr,y_test_data)

### Random Forest Classifier - Confusion Matrix and Accuracy score

In [50]:
from sklearn.ensemble import RandomForestClassifier
randomregressor = RandomForestClassifier(n_estimators=200,criterion='entropy',max_depth=5, random_state=1)
randomregressor.fit(X_train_data,y_train_Data)

RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=200,
                       random_state=1)

In [51]:
y_pred_rand = randomregressor.predict(X_test_data)
rand_confusionMatrix = confusion_matrix(y_pred_rand,y_test_data)
rand_Accuracy = accuracy_score(y_pred_rand,y_test_data)

### Confusion Matrix and Accuracy Score for Each Algorithms

In [54]:
numpy_data = np.array([[logistic_confusionMatrix,logistic_Accuracy],[knn_confusionMatrix,knn_Accuracy],[svr_confusionMatrix,svr_Accuracy],[rand_confusionMatrix,rand_Accuracy]],dtype=object)
df = pd.DataFrame(data=numpy_data, index=["Logistic Regression", "KNN Classifier","Support Vector Classifier","Random Forest Classifier"], columns=["Confusion Matrix","Accuracy Score"])
print(df.sort_values(by=["Accuracy Score"]))

                                 Confusion Matrix Accuracy Score
KNN Classifier             [[247, 12], [19, 140]]       0.925837
Support Vector Classifier  [[247, 12], [19, 140]]       0.925837
Logistic Regression         [[252, 4], [14, 148]]       0.956938
Random Forest Classifier    [[264, 12], [2, 140]]       0.966507


## Final Output

In [55]:
output = pd.DataFrame({'PassengerId': gs.PassengerId, 'Survived': y_pred_rand})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
